# asyncio

In [3]:
import asyncio
import time

## 结合多进程和 Asyncio 以提高性能

In [4]:
def sum_to_num(final_num: int) -> int:
    start = time.monotonic()

    result = 0
    for i in range(0, final_num+1, 1):
        result += i

    print(f"The method with {final_num} completed in {time.monotonic() - start:.2f} second(s).")
    return result

## 多进程

In [ ]:
from multiprocessing import Process
def main():
    # We initialize the two processes with two parameters, from largest to smallest
    process_a = Process(target=sum_to_num, args=(200_000_000,))
    process_b = Process(target=sum_to_num, args=(50_000_000,))

    # And then let them start executing
    process_a.start()
    process_b.start()

    # Note that the join method is blocking and gets results sequentially
    start_a = time.monotonic()
    process_a.join()
    print(f"Process_a completed in {time.monotonic() - start_a:.2f} seconds")

    # Because when we wait process_a for join. The process_b has joined already.
    # so the time counter is 0 seconds.
    start_b = time.monotonic()
    process_b.join()
    print(f"Process_b completed in {time.monotonic() - start_b:.2f} seconds")

创建并启动多个进程，调用每个进程的start和join方法。但是，这里存在一些问题：

1. join 方法不能返回任务执行的结果。
2. join 方法阻塞主进程并按顺序执行它。

## 进程池

In [6]:
from multiprocessing import Pool
def main():
    with Pool() as pool:
        result_a = pool.apply(sum_to_num, args=(200_000_000,))
        result_b = pool.apply(sum_to_num, args=(50_000_000,))

        print(f"sum_to_num with 200_000_000 got a result of {result_a}.")
        print(f"sum_to_num with 50_000_000 got a result of {result_b}.")

Pool 的 apply 方法是同步的，这意味着您必须等待之前的 apply 任务完成才能开始执行下一个 apply 任务。

使用 apply_async 方法异步创建任务。但是同样，您需要使用 get 方法来阻塞地获取结果。它让我们回到 join 方法的问题：

In [7]:
def main():
    with Pool() as pool:
        result_a = pool.apply_async(sum_to_num, args=(200_000_000,))
        result_b = pool.apply_async(sum_to_num, args=(50_000_000,))

        print(f"sum_to_num with 200_000_000 got a result of {result_a.get()}.")
        print(f"sum_to_num with 50_000_000 got a result of {result_b.get()}.")

## ProcessPoolExecutor
那么，使用 concurrent.futures.ProcesssPoolExecutor 来执行我们的 CPU 绑定任务呢？

In [9]:
from concurrent .futures import ProcessPoolExecutor
def main():
    with ProcessPoolExecutor() as executor:
        numbers = [200_000_000, 50_000_000]
        for result in executor.map(sum_to_num, numbers):
            print(f"sum_to_num got a result which is {result}.")

一切看起来都很棒，并且就像 asyncio.as_completed 一样被调用。但是它们仍按启动顺序获取。可以使用 asyncio 来处理 IO-bound 任务，它的 run_in_executor 方法可以像 asyncio 一样调用多进程任务。不仅统一了并发和并行的API，还解决了我们上面遇到的各种问题：

## asyncio 的 run_in_executor

In [10]:
async def main():
    loop = asyncio.get_running_loop()
    tasks = []

    with ProcessPoolExecutor() as executor:
        for number in [200_000_000, 50_000_000]:
            tasks.append(loop.run_in_executor(executor, sum_to_num, number))
        
        # Or we can just use the method asyncio.gather(*tasks)
        for done in asyncio.as_completed(tasks):
            result = await done
            print(f"sum_to_num got a result which is {result}")